In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 68.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e812a8ea0c8060d9a2301878ac2ce76cf961888662325cf74c3c160de4ced77c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("walmart_data").getOrCreate()

### Create the schema of the streamed files (check the column names and types from the CSV files)

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , DateType ,DoubleType

Schema = StructType([StructField('id', IntegerType(), True),
                           StructField('Date', DateType(), True),
                           StructField('Open', DoubleType(), True),
                           StructField('High', DoubleType(), True),
                           StructField('Low', DoubleType(), True),
                           StructField('Close', DoubleType(), True),
                           StructField('Adj Close', DoubleType(), True),
                           StructField('Volume', IntegerType(), True),
                           
                           
                           ])

### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [5]:
df=spark.readStream.csv("/content/sample_data/kospi",Schema)

### Make sure the sataframe is streaming the files from the folder

In [7]:
df.isStreaming

True

### Create a stream writer into memory and specify the query name "stock:

In [8]:
sr=df.writeStream.format('memory').queryName('stock')

### Start the write stream and make sure it works (read all columns from the table)

In [ ]:
sr.start()

In [11]:
spark.sql("SELECT* FROM stock").show()

+----+----------+------------+------------+------------+------------+------------+------+
|  id|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+----+----------+------------+------------+------------+------------+------------+------+
|null|      null|        null|        null|        null|        null|        null|  null|
| 240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|
| 241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|
| 242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656|
| 243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|
| 244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|
| 245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|
| 246|2000-12-13|27651.599609|     29286.5|27651.599609|28469.099609|26903.234375|270385|
| 247|2000

### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [21]:
df=df.dropna(how="all")
df=df.withColumn("diff",df.High-df.Low)


### Create a new write stream using the new generated dataframe and call the generate table "modified_data"

In [22]:
sr2=df.writeStream.format('memory').queryName("modified_data")

In [23]:
sr2.start()

In [24]:
spark.sql("SELECT* FROM modified_data ").show()

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| id|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|  995.201172000001|
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791| 710.8007819999984|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656| 533.1015620000035|
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964| 1066.298827999999|
|244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|1208.4003909999992|
|245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|  639.798827999999|
|246|2000-12-13|27651.599609

### Write the generated data into files instead of the memory. 

In [29]:
sr3=df.writeStream.format('csv').outputMode('append').option("path", "OutStream/").option("checkpointLocation", "new_file") 
sr3.start()

### Stop the query. Now, try reading the generated parquet files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [35]:
finalDF=spark.read.csv('/content/OutStream/',schema=Schema,header=True)
finalDF.show()


+---+----------+------------+------------+------------+------------+------------+------+
| id|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+---+----------+------------+------------+------------+------------+------------+------+
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656|
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|
|244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|
|245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|
|246|2000-12-13|27651.599609|     29286.5|27651.599609|28469.099609|26903.234375|270385|
|247|2000-12-14|28469.099609|29784.099609|28291.300781|28362.400391| 26802.40625|256317|
|248|2000-12-15|28362.400391|28895.599609|27793.800781|27935.900391|26399.361328|108886|
|249|2000-12-18|     

### Sort the dataframe based on the ID

In [36]:
finalDFSorted = finalDF.sort('ID')
finalDFSorted.show()

+---+----------+------------+------------+------------+------------+------------+------+
| id|      Date|        Open|        High|         Low|       Close|   Adj Close|Volume|
+---+----------+------------+------------+------------+------------+------------+------+
|  0|2000-01-04|22817.900391|25696.800781|22817.900391|24879.300781|23510.880859|108745|
|  1|2000-01-05|24523.900391|26229.900391|23670.900391|24417.300781|23074.294922|175990|
|  2|2000-01-06|24381.699219|24666.099609|22746.800781|22817.900391|21562.865234| 71746|
|  3|2000-01-07|     22036.0|24879.300781|     22036.0|23884.199219|22570.513672|120984|
|  4|2000-01-10|24879.300781|25519.099609|23813.099609|24061.900391|22738.439453|151371|
|  5|2000-01-11|     24168.5|     25021.5|23955.199219|24239.599609|22906.365234| 95943|
|  6|2000-01-12|     24168.5|24452.800781|23457.599609|23670.900391|22368.947266| 61899|
|  7|2000-01-13|23670.900391|24132.900391|23102.199219|23244.400391| 21965.90625| 57538|
|  8|2000-01-14|23457